<a href="https://colab.research.google.com/github/welcomeglory/Python/blob/master/0808_Heart_Disease_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
#전처리
import pandas as pd
import numpy as np
#시각화
import matplotlib.pyplot as plt
import seaborn as sns

In [50]:
df = pd.read_csv("/content/drive/MyDrive/세종교육/Heart Disease Prediction/Heart_Disease_Prediction.csv")

In [51]:
df.head()

,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,70,1,4,130,322,0,2,109,0,2.4,2,3,3,Presence
1,67,0,3,115,564,0,2,160,0,1.6,2,0,7,Absence
2,57,1,2,124,261,0,0,141,0,0.3,1,0,7,Presence
3,64,1,4,128,263,0,0,105,1,0.2,2,1,7,Absence
4,74,0,2,120,269,0,2,121,1,0.2,1,1,3,Absence


In [52]:
df= df.replace({'Heart Disease': {'Presence': 1, 'Absence': 0}})

'Presence'라는 문자열을 1로, 'Absence'라는 문자열을 0으로 변경

In [53]:
df.info()
# 결측치 없음 - NULL값이 없다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      270 non-null    int64  
 1   Sex                      270 non-null    int64  
 2   Chest pain type          270 non-null    int64  
 3   BP                       270 non-null    int64  
 4   Cholesterol              270 non-null    int64  
 5   FBS over 120             270 non-null    int64  
 6   EKG results              270 non-null    int64  
 7   Max HR                   270 non-null    int64  
 8   Exercise angina          270 non-null    int64  
 9   ST depression            270 non-null    float64
 10  Slope of ST              270 non-null    int64  
 11  Number of vessels fluro  270 non-null    int64  
 12  Thallium                 270 non-null    int64  
 13  Heart Disease            270 non-null    int64  
dtypes: float64(1), int64(13)
m

*target* 변수의 비율 확인

In [54]:
df['Heart Disease'].value_counts()/len(df)
#불균형은 보이지 않음

,count
Heart Disease,
0,0.555556
1,0.444444


In [55]:
X= df.drop(["Heart Disease"],axis=1)
y= df[["Heart Disease"]]

In [56]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [57]:
# 정규화, 인코딩
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      270 non-null    int64  
 1   Sex                      270 non-null    int64  
 2   Chest pain type          270 non-null    int64  
 3   BP                       270 non-null    int64  
 4   Cholesterol              270 non-null    int64  
 5   FBS over 120             270 non-null    int64  
 6   EKG results              270 non-null    int64  
 7   Max HR                   270 non-null    int64  
 8   Exercise angina          270 non-null    int64  
 9   ST depression            270 non-null    float64
 10  Slope of ST              270 non-null    int64  
 11  Number of vessels fluro  270 non-null    int64  
 12  Thallium                 270 non-null    int64  
dtypes: float64(1), int64(12)
memory usage: 27.5 KB


## 수치형과 범주형 변수 나누어 인코딩, 스케일링 진행

In [141]:
numerical_ix = X.columns[[0, 3, 4, 5] ]
categorical_ix = X.columns[[1, 2, 5, 6, 8,10,11,12 ]]

In [142]:
t = [('cat', OneHotEncoder(handle_unknown = "ignore"), categorical_ix), ('num', StandardScaler(), numerical_ix)]
col_transform = ColumnTransformer(transformers=t)

In [143]:
X_train_std = col_transform.fit_transform(X_train)
X_test_std = col_transform.transform(X_test)

In [144]:
X_train_std

array([[ 0.        ,  1.        ,  1.        , ..., -0.67478847,
        -1.05780979, -0.41763453],
       [ 1.        ,  0.        ,  0.        , ...,  1.67028286,
        -0.90567196, -0.41763453],
       [ 0.        ,  1.        ,  0.        , ..., -1.84732413,
        -0.50631017, -0.41763453],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  4.01535418,
         0.7488269 ,  2.394438  ],
       [ 0.        ,  1.        ,  0.        , ..., -1.2610563 ,
        -0.98174087, -0.41763453],
       [ 0.        ,  1.        ,  0.        , ..., -1.84732413,
         0.95801641, -0.41763453]])

## Modlening and Tuning

In [145]:
!pip install catboost

In [146]:
#classifier 설정
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [147]:
#튜닝 방법:grid
from sklearn.model_selection import GridSearchCV

In [148]:
#model 정의
models = {
    # 'RandomForest': (RandomForestClassifier(),
    #              {
    #                  'n_estimators': [50, 100],
    #                  'max_depth': [2, 20],
    #                  'min_samples_split':[5, 100],
    #              }),
    # 'LightGBM': (LGBMClassifier(),
    #             {
    #                  'n_estimators': [50, 100],
    #                  'max_depth': [2, 20],
    #                  'min_child_samples':[5, 100],
    #             }),
    #LGBM 계속 커널 오류
    'GB': (GradientBoostingClassifier(),
                 {
                     'n_estimators': [50, 100],
                     'max_depth': [2, 20],
                     'min_samples_split':[5, 100],
                 }),
    'ExtraTree': (ExtraTreesClassifier(),
                  {
                     'n_estimators': [50, 100],
                     'max_depth': [2, 20],
                     'min_samples_split':[5, 100],
                 }),

    'Catboost': (CatBoostClassifier(),
                 {
                    'depth': [4, 5, 6],
                    'leaf_estimation_iterations': [10]
                 })
}

In [149]:
models.items()

dict_items([('GB', (GradientBoostingClassifier(), {'n_estimators': [50, 100], 'max_depth': [2, 20], 'min_samples_split': [5, 100]})), ('ExtraTree', (ExtraTreesClassifier(), {'n_estimators': [50, 100], 'max_depth': [2, 20], 'min_samples_split': [5, 100]})), ('Catboost', (<catboost.core.CatBoostClassifier object at 0x79a630248a90>, {'depth': [4, 5, 6], 'leaf_estimation_iterations': [10]}))])

In [150]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate

In [151]:
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [152]:
model_results = []

# 모델별로 튜닝 및 적합
for model_name, (model, param_grid) in models.items():
    print(f"Start {model_name} Training!")
    search = GridSearchCV(model, param_grid, scoring='accuracy',  n_jobs=-1)
    search.fit(X_train_std, y_train)

    # 최적의 모델과 하이퍼파라미터 출력
    best_model =search.best_estimator_
    best_params =search.best_params_
    print(f"Best Parameters: {best_params}")

    # 테스트 데이터에 대한 예측 및 평가
    y_pred = best_model.predict(X_test_std)
    f1 = f1_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    c_m = confusion_matrix(y_test, y_pred)
    print(f"F1 score: {f1}\n")
    print(f"Accuracy: {accuracy}\n")
    print(f"Confusion Matrix: {c_m}\n")


    # 모델 이름과 accuracy 결과 저장
    model_results.append((model_name, accuracy, f1))

Start GB Training!


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Best Parameters: {'max_depth': 2, 'min_samples_split': 100, 'n_estimators': 50}
F1 score: 0.7999999999999999

Accuracy: 0.8235294117647058

Confusion Matrix: [[32  8]
 [ 4 24]]

Start ExtraTree Training!


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Best Parameters: {'max_depth': 2, 'min_samples_split': 5, 'n_estimators': 100}
F1 score: 0.7169811320754718

Accuracy: 0.7794117647058824

Confusion Matrix: [[34  6]
 [ 9 19]]

Start Catboost Training!
0:	learn: 0.6672554	total: 2.27ms	remaining: 2.27s
1:	learn: 0.6449018	total: 4.5ms	remaining: 2.24s
2:	learn: 0.6250323	total: 5.67ms	remaining: 1.88s
3:	learn: 0.6065856	total: 8.37ms	remaining: 2.08s
4:	learn: 0.5872743	total: 11.3ms	remaining: 2.25s
5:	learn: 0.5699934	total: 13.1ms	remaining: 2.17s
6:	learn: 0.5552052	total: 14.9ms	remaining: 2.11s
7:	learn: 0.5399596	total: 16.8ms	remaining: 2.08s
8:	learn: 0.5250059	total: 18.3ms	remaining: 2.01s
9:	learn: 0.5141189	total: 19.9ms	remaining: 1.97s
10:	learn: 0.5040063	total: 21.2ms	remaining: 1.9s
11:	learn: 0.4944745	total: 22.7ms	remaining: 1.87s
12:	learn: 0.4820812	total: 25.1ms	remaining: 1.9s
13:	learn: 0.4723321	total: 26.3ms	remaining: 1.85s
14:	learn: 0.4623104	total: 27.6ms	remaining: 1.81s
15:	learn: 0.4531634	total: 29.

In [153]:
model_results

[('GB', 0.8235294117647058, 0.7999999999999999),
 ('ExtraTree', 0.7794117647058824, 0.7169811320754718),
 ('Catboost', 0.7647058823529411, 0.7142857142857143)]

In [154]:
models

{'GB': (GradientBoostingClassifier(),
  {'n_estimators': [50, 100],
   'max_depth': [2, 20],
   'min_samples_split': [5, 100]}),
 'ExtraTree': (ExtraTreesClassifier(),
  {'n_estimators': [50, 100],
   'max_depth': [2, 20],
   'min_samples_split': [5, 100]}),
 'Catboost': (<catboost.core.CatBoostClassifier at 0x79a630248a90>,
  {'depth': [4, 5, 6], 'leaf_estimation_iterations': [10]})}

In [156]:
from sklearn.ensemble import VotingClassifier

# VotingClassifier 설정
voting_classifier = VotingClassifier(estimators=voting_estimators, voting='soft')


In [157]:
voting_classifier.fit(X_train_std, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


0:	learn: 0.6672554	total: 1.47ms	remaining: 1.47s
1:	learn: 0.6449018	total: 3.34ms	remaining: 1.67s
2:	learn: 0.6250323	total: 4.8ms	remaining: 1.6s
3:	learn: 0.6065856	total: 6.13ms	remaining: 1.52s
4:	learn: 0.5872743	total: 7.57ms	remaining: 1.51s
5:	learn: 0.5699934	total: 8.85ms	remaining: 1.47s
6:	learn: 0.5552052	total: 10.2ms	remaining: 1.45s
7:	learn: 0.5399596	total: 11.6ms	remaining: 1.44s
8:	learn: 0.5250059	total: 13.1ms	remaining: 1.44s
9:	learn: 0.5141189	total: 14.4ms	remaining: 1.42s
10:	learn: 0.5040063	total: 15.4ms	remaining: 1.38s
11:	learn: 0.4944745	total: 16.7ms	remaining: 1.38s
12:	learn: 0.4820812	total: 18ms	remaining: 1.37s
13:	learn: 0.4723321	total: 19ms	remaining: 1.34s
14:	learn: 0.4623104	total: 20.4ms	remaining: 1.34s
15:	learn: 0.4531634	total: 21.6ms	remaining: 1.33s
16:	learn: 0.4431537	total: 23ms	remaining: 1.33s
17:	learn: 0.4372965	total: 24.3ms	remaining: 1.33s
18:	learn: 0.4258372	total: 25.7ms	remaining: 1.32s
19:	learn: 0.4172408	total: 27

VotingClassifier(estimators=[('RandomForest',
                              RandomForestClassifier(max_depth=2,
                                                     min_samples_split=5)),
                             ('GB',
                              GradientBoostingClassifier(max_depth=2,
                                                         min_samples_split=100,
                                                         n_estimators=50)),
                             ('Catboost',
                              <catboost.core.CatBoostClassifier object at 0x79a654390c10>)],
                 voting='soft')

In [158]:
y_pred = voting_classifier.predict(X_test_std)
f1 = f1_score(y_test, y_pred)
c_m = confusion_matrix(y_test, y_pred)
print(f"F1 score: {f1}\n")
print(f"Accuracy: {accuracy}\n")
print(f"Confusion Matrix: {c_m}\n")

F1 score: 0.736842105263158

Accuracy: 0.7647058823529411

Confusion Matrix: [[32  8]
 [ 7 21]]



In [160]:
# hard VotingClassifier
voting_classifier = VotingClassifier(estimators=voting_estimators, voting='hard')
voting_classifier.fit(X_train_std, y_train)
y_pred = voting_classifier.predict(X_test_std)
f1 = f1_score(y_test, y_pred)
c_m = confusion_matrix(y_test, y_pred)
print(f"F1 score: {f1}\n")
print(f"Accuracy: {accuracy}\n")
print(f"Confusion Matrix: {c_m}\n")

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


0:	learn: 0.6672554	total: 1.3ms	remaining: 1.3s
1:	learn: 0.6449018	total: 3.07ms	remaining: 1.53s
2:	learn: 0.6250323	total: 4.42ms	remaining: 1.47s
3:	learn: 0.6065856	total: 5.79ms	remaining: 1.44s
4:	learn: 0.5872743	total: 7.01ms	remaining: 1.4s
5:	learn: 0.5699934	total: 8.32ms	remaining: 1.38s
6:	learn: 0.5552052	total: 9.59ms	remaining: 1.36s
7:	learn: 0.5399596	total: 10.8ms	remaining: 1.34s
8:	learn: 0.5250059	total: 12.1ms	remaining: 1.33s
9:	learn: 0.5141189	total: 13.5ms	remaining: 1.34s
10:	learn: 0.5040063	total: 14.5ms	remaining: 1.3s
11:	learn: 0.4944745	total: 15.8ms	remaining: 1.3s
12:	learn: 0.4820812	total: 17ms	remaining: 1.29s
13:	learn: 0.4723321	total: 18ms	remaining: 1.27s
14:	learn: 0.4623104	total: 19.3ms	remaining: 1.27s
15:	learn: 0.4531634	total: 20.5ms	remaining: 1.26s
16:	learn: 0.4431537	total: 21.7ms	remaining: 1.25s
17:	learn: 0.4372965	total: 22.9ms	remaining: 1.25s
18:	learn: 0.4258372	total: 24.8ms	remaining: 1.28s
19:	learn: 0.4172408	total: 26.